<a href="https://colab.research.google.com/github/sharop/CD3001B/blob/main/clase_1/Tema-1_4-1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de librerias

In [5]:
#!apt-get install r-base
#!pip install rpy2
!pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Preparación de ambiente

In [1]:
# Download the data and assets that this notebook uses
import urllib.request

url_prefix = "https://github.com/jmbejara/comp-econ-sp19/blob/master/lectures/4-23_Panel_Data/"
files = [
    "15_2_Wooldridge_Intro.png",
    "lme.dta",
    "Example_10_4_Wooldridge_Panel.png",
    "Example_10_5_Wooldridge_Panel.png",
    "fixed_and_random_effects.do",
    "St11-12_jtrain1.dta",
    ]
for file in files:
    url = url_prefix + file + "?raw=true"
    urllib.request.urlretrieve(url, f'./{file}')

El codigo que se encuentra en las celdas previas solo es para Google Colab.
El siguiente codigo es generico.

#Exploración del ambiente


In [6]:
import numpy as np
import pandas as pd
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import linearmodels
from statsmodels.sandbox.regression.predstd import wls_prediction_std

np.random.seed(9876789)

In [10]:
# Muestra simple con datos faltantes.
nsample = 100
x = np.linspace(0, 10, 100)
X = np.column_stack((x, x**2))
beta = np.array([0.1, 10])
e = np.random.normal(size=nsample)
alpha = 1
y = alpha + X @ beta + e
X[3,1] = np.nan # agregamos datos faltantes

###Statsmodel con el API estámdar.

In [11]:
df = pd.DataFrame(X, columns=['x1', 'x2'])
df['y'] = y
df.head()

,x1,x2,y
0,0.00000,0.000000,0.159484
1,0.10101,0.010203,2.709627
2,0.20202,0.040812,1.893866
3,0.30303,NaN,1.440418
4,0.40404,0.163249,5.607196


In [12]:
rhs = sm.add_constant(df[['x1', 'x2']])
lhs = df['y']
res = sm.OLS(lhs, rhs, missing='drop').fit()

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [13]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.955e+06
Date:                Tue, 14 Feb 2023   Prob (F-statistic):          1.09e-236
Time:                        17:19:47   Log-Likelihood:                -145.23
No. Observations:                  99   AIC:                             296.5
Df Residuals:                      96   BIC:                             304.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4086      0.324      4.341      0.000       0.765       2.053
x1            -0.0654      0.148     -0.441      0.660      -0.360       0.229
x2            10.0123      0.014    702.595      0.000       9.984      10.041
==============================================================================
Omnibus:                        1.981   Durbin-Watson:                   2.228
Prob(Omnibus):                  0.371   Jarque-Bera (JB):                1.764
Skew:                           0.209   Prob(JB):                        0.414
Kurtosis:                       2.498   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
res.params

const     1.408565
x1       -0.065398
x2       10.012289
dtype: float64

### Statsmodels utilizando el API de formula.
Estilo R. Hay qeue revisar que el paquete `import statsmodels.formula.api as smf`

In [15]:
results = smf.ols(formula="y ~ x1 + x2", data=df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.955e+06
Date:                Tue, 14 Feb 2023   Prob (F-statistic):          1.09e-236
Time:                        17:28:33   Log-Likelihood:                -145.23
No. Observations:                  99   AIC:                             296.5
Df Residuals:                      96   BIC:                             304.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4086      0.324      4.341      0.000       0.765       2.053
x1            -0.0654      0.148     -0.441      0.660      -0.360       0.229
x2            10.0123      0.014    702.595      0.000       9.984      10.041
==============================================================================
Omnibus:                        1.981   Durbin-Watson:                   2.228
Prob(Omnibus):                  0.371   Jarque-Bera (JB):                1.764
Skew:                           0.209   Prob(JB):                        0.414
Kurtosis:                       2.498   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#1.4. Modelo de efectos fijos

In statsmodels

Here is a quick example of the syntax from statsmodels. This is taken from here: http://statsmodels.sourceforge.net/devel/mixed_linear.html

#1.5 Modelo de efectos aleatorios